In [9]:
import os
import sys
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

notebook_name = 'dimensionality_reduction.ipynb'

current_dir = os.path.dirname(os.path.abspath(notebook_name))
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))


sys.path.insert(1, parent_dir)
from src.data_processer import TelecomDataProcessor

In [2]:
# Replace 'your_conn_params' with your actual connection parameters
conn_params = {
    'host': 'localhost',
    'port': 5432,
    'user': 'ekru',
    'password': 'ekram12345',
    'database': 'tellco',
}

telecom_data_processor = TelecomDataProcessor(conn_params)


In [3]:
df = telecom_data_processor.load_dataset()

/home/ekram/Desktop/week-1/user_analytics_in_the_telecommunication_industry/src/data_processer.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [4]:
cleaned_df = telecom_data_processor.clean_dataset(df)

/home/ekram/Desktop/week-1/user_analytics_in_the_telecommunication_industry/src/data_processer.py:88: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


           Bearer Id            Start  Start ms              End     End ms  \
0       1.311448e+19   4/4/2019 12:01  770.0000  4/25/2019 14:35  662.00000   
1       1.311448e+19   4/9/2019 13:04  235.0000   4/25/2019 8:15  606.00000   
2       1.311448e+19   4/9/2019 17:42    1.0000  4/25/2019 11:58  652.00000   
3       1.311448e+19   4/10/2019 0:31  486.0000   4/25/2019 7:36  171.00000   
4       1.311448e+19  4/12/2019 20:10  565.0000  4/25/2019 10:40  954.00000   
...              ...              ...       ...              ...        ...   
149996  7.277826e+18   4/29/2019 7:28  451.0000   4/30/2019 6:02  214.00000   
149997  7.349883e+18   4/29/2019 7:28  483.0000  4/30/2019 10:41  187.00000   
149998  1.311448e+19   4/29/2019 7:28  283.0000  4/30/2019 10:46  810.00000   
149999  1.311448e+19   4/29/2019 7:28  696.0000  4/30/2019 10:40  327.00000   
150000  1.013887e+19             None  499.1882             None  498.80088   

           Dur. (ms)          IMSI  MSISDN/Number  

In [5]:
telecom_data_processor.overview_analysis(cleaned_df)

          BEARER_ID       START_MS         END_MS     DUR._(MS)          IMSI  \
count  1.500010e+05  150001.000000  150001.000000  1.500010e+05  1.500010e+05   
mean   1.013887e+19     499.188200     498.800880  1.046086e+05  2.082016e+14   
std    2.883600e+18     288.610872     288.096693  8.103735e+04  2.144722e+10   
min    6.917538e+18       0.000000       0.000000  7.142000e+03  2.040471e+14   
25%    7.349883e+18     250.000000     251.000000  5.744200e+04  2.082014e+14   
50%    7.349883e+18     499.000000     500.000000  8.639900e+04  2.082015e+14   
75%    1.304243e+19     749.000000     750.000000  1.324300e+05  2.082018e+14   
max    1.318654e+19     999.000000     999.000000  1.859336e+06  2.140743e+14   

       MSISDN/NUMBER          IMEI  AVG_RTT_DL_(MS)  AVG_RTT_UL_(MS)  \
count   1.500010e+05  1.500010e+05    150001.000000    150001.000000   
mean    4.188282e+10  4.847455e+13       109.795706        17.662883   
std     2.438731e+12  2.237359e+13       559.342620   

In [13]:
numberic_columns = cleaned_df.select_dtypes(include=['float64', 'int64']).columns
data = cleaned_df[numeric_columns]
# Fill missing values with the mean of each column
data = data.fillna(data.mean())
# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Perform PCA
pca = PCA()
principal_components = pca.fit_transform(scaled_data)

# Explained Variance
explained_variance_ratio = pca.explained_variance_ratio_

# Cumulative Explained Variance
cumulative_explained_variance = explained_variance_ratio.cumsum()

# Identify the optimal number of components based on explained variance
# Choose a threshold (e.g., 95% variance explained)
n_components = (cumulative_explained_variance < 0.95).sum() + 1

# Redo PCA with the optimal number of components
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
columns = [f'PC{i}' for i in range(1, n_components + 1)]
principal_cleaned_df = pd.DataFrame(data=principal_components, columns=columns)

# Optional: Attach the principal components back to the original DataFrame if needed
df_with_pca = pd.concat([cleaned_df, principal_cleaned_df], axis=1)

# Display the results
print("Explained Variance Ratio:")
print(explained_variance_ratio)
print("\nCumulative Explained Variance:")
print(cumulative_explained_variance)
print("\nNumber of Components Chosen:")
print(n_components)

Explained Variance Ratio:
[1.50523960e-01 5.90098324e-02 4.92136657e-02 4.00474337e-02
 3.98640783e-02 3.50191014e-02 3.37320991e-02 3.16937459e-02
 2.92643272e-02 2.80166130e-02 2.28545044e-02 2.24300750e-02
 2.10818912e-02 2.03892325e-02 2.02694246e-02 2.02033712e-02
 2.01547033e-02 2.00937297e-02 2.00270750e-02 1.99896737e-02
 1.99647432e-02 1.99474466e-02 1.98650423e-02 1.97707459e-02
 1.97536616e-02 1.96945377e-02 1.91796017e-02 1.86460141e-02
 1.75565512e-02 1.48730121e-02 1.45913179e-02 1.38022616e-02
 1.23392246e-02 1.06247413e-02 1.03186821e-02 9.51051907e-03
 7.64480750e-03 6.87036431e-03 5.72345735e-03 5.51272425e-03
 3.52916985e-03 3.08737381e-03 1.61380951e-03 1.27536446e-03
 3.29492252e-04 7.86035271e-05 1.81934232e-05 1.49925514e-13
 3.29640789e-33 3.02458593e-33]

Cumulative Explained Variance:
[0.15052396 0.20953379 0.25874746 0.29879489 0.33865897 0.37367807
 0.40741017 0.43910392 0.46836824 0.49638486 0.51923936 0.54166944
 0.56275133 0.58314056 0.60340998 0.62361336